In [1]:
from time import sleep
import random


# Para trabajar con ficheros
# -----------------------------------------------------------------------
import os
import re

import requests
import pandas as pd
import json

from tqdm import tqdm

# ---------------------------------------
# Path
import sys
sys.path.append('../')

import warnings
#warnings.filterwarnings('ignore')

import fastf1

from src.support import get_dotd, get_df_dotd, get_df_circuit, get_df_drivers, get_df_constructors, transform_df_results, get_number_of_races_in_season

### Driver of the day

In [2]:
try:
    df_dotd = pd.read_csv('../data/dotd.csv')
except FileNotFoundError:
    soup = get_dotd()
    df_dotd = get_df_dotd(soup)
    df_dotd.to_csv('../data/dotd.csv', index = False)

### Circuits (2023)

In [3]:
try:
    df_circuit = pd.read_csv('../data/circuit.csv')
except FileNotFoundError:
    df_circuit = get_df_circuit(2023)
    df_circuit.to_csv('../data/circuit.csv', index = False)

### Drivers (2023)

In [4]:
try:
    df_drivers = pd.read_csv('../data/drivers.csv')
except FileNotFoundError:
    df_drivers = get_df_drivers(2023)
    df_drivers.to_csv('../data/drivers.csv', index = False)

### Constructors (2023)

In [5]:
try:
    df_constructors = pd.read_csv('../data/constructors.csv')
except FileNotFoundError:
    df_constructors = get_df_constructors(2023)
    df_constructors.to_csv('../data/constructors.csv', index = False)

### Races & results

In [2]:
year = 2023

number_of_races = get_number_of_races_in_season(year)
results_list = []
races_list = []

for rnd in tqdm(range(1, number_of_races + 1)):

    race_id = str(year) + '_' + str(rnd)

    url = f"http://ergast.com/api/f1/{str(year)}/{str(rnd)}/results.json"
    response = requests.get(url)

    if response.status_code == 200:

        content = response.json()

        # Pop results
        results = content['MRData']['RaceTable']['Races'][0].pop('Results')
        # Build a dataframe for results
        df_result = transform_df_results(results, race_id)
        # Store dataframe
        results_list.append(df_result)

        # Build a dataframe for races
        df_race = pd.DataFrame(content['MRData']['RaceTable']['Races'][0]).loc['circuitId']
        df_race['race_id'] = race_id
        # Store dataframe
        races_list.append(df_race)

    else:
        print(f"Error: {response.status_code}")

# Put together all races
df_results = pd.concat(results_list)
df_races = pd.concat(races_list, axis=1).T.reset_index(drop=True)
# Rename and sort columns
df_races.rename(columns={'Circuit': 'circuit_id'}, inplace=True)
df_races = df_races[['race_id', 'circuit_id', 'raceName', 'season', 'round', 'url', 'date', 'time']]

100%|██████████| 22/22 [00:34<00:00,  1.56s/it]


In [3]:
df_results

,race_id,position,positionText,points,grid,laps,status,driver_id,constructor_id,delta_pos,time
0,2023_1,1,1,25,1,57,Finished,max_verstappen,red_bull,0,1:33:56.736
1,2023_1,2,2,18,2,57,Finished,perez,red_bull,0,+11.987
2,2023_1,3,3,15,5,57,Finished,alonso,aston_martin,2,+38.637
3,2023_1,4,4,12,4,57,Finished,sainz,ferrari,0,+48.052
4,2023_1,5,5,10,7,57,Finished,hamilton,mercedes,2,+50.977
...,...,...,...,...,...,...,...,...,...,...,...
15,2023_22,16,16,0,20,58,Finished,sargeant,williams,4,+1:27.791
16,2023_22,17,17,0,19,58,Finished,zhou,alfa,2,+1:29.422
17,2023_22,18,18,0,16,57,Retired,sainz,ferrari,-2,NaN
18,2023_22,19,19,0,18,57,+1 Lap,bottas,alfa,-1,NaN


In [4]:
df_races

,race_id,circuit_id,raceName,season,round,url,date,time
0,2023_1,bahrain,Bahrain Grand Prix,2023,1,https://en.wikipedia.org/wiki/2023_Bahrain_Gra...,2023-03-05,15:00:00Z
1,2023_2,jeddah,Saudi Arabian Grand Prix,2023,2,https://en.wikipedia.org/wiki/2023_Saudi_Arabi...,2023-03-19,17:00:00Z
2,2023_3,albert_park,Australian Grand Prix,2023,3,https://en.wikipedia.org/wiki/2023_Australian_...,2023-04-02,05:00:00Z
3,2023_4,baku,Azerbaijan Grand Prix,2023,4,https://en.wikipedia.org/wiki/2023_Azerbaijan_...,2023-04-30,11:00:00Z
4,2023_5,miami,Miami Grand Prix,2023,5,https://en.wikipedia.org/wiki/2023_Miami_Grand...,2023-05-07,19:30:00Z
5,2023_6,monaco,Monaco Grand Prix,2023,6,https://en.wikipedia.org/wiki/2023_Monaco_Gran...,2023-05-28,13:00:00Z
6,2023_7,catalunya,Spanish Grand Prix,2023,7,https://en.wikipedia.org/wiki/2023_Spanish_Gra...,2023-06-04,13:00:00Z
7,2023_8,villeneuve,Canadian Grand Prix,2023,8,https://en.wikipedia.org/wiki/2023_Canadian_Gr...,2023-06-18,18:00:00Z
8,2023_9,red_bull_ring,Austrian Grand Prix,2023,9,https://en.wikipedia.org/wiki/2023_Austrian_Gr...,2023-07-02,13:00:00Z
9,2023_10,silverstone,British Grand Prix,2023,10,https://en.wikipedia.org/wiki/2023_British_Gra...,2023-07-09,14:00:00Z
